In [1395]:
import httplib2
import pandas as pd
import glob
import datetime as dt
import locale
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')  # Устанавливаем поддержку русского языка

pd.options.display.max_seq_items = 2000
import pandas as pd
import gspread
from googleapiclient.discovery import build
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [1396]:
df = pd.read_csv(glob.glob('/Users/iliya/Egor/iTCODE/reports/business-supervisor_issues_2022-**-**.csv')[0])

In [1397]:
df = df.drop(['Weight', 'Closed At (UTC)', 'Due Date', 'Assignee Username',
              'Assignee', 'Author Username', 'Author', 'State', 'Locked',
              'Confidential', 'Created At (UTC)', 'Updated At (UTC)', 'Time Estimate', 'Time Spent'], axis=1)

In [1398]:
report_date = dt.date.today().strftime("%d %B (%A)")

In [1399]:
report_date

'07 декабря (среда)'

In [1400]:
df = df.rename(columns={
    'Issue ID': '№',
    'URL': 'Ссылка',
    'Title': 'Заголовок',
    'Description': f'Описание задач за {report_date}',
    'Milestone': 'Дата',
    'Labels': 'Тэги'
})

In [1401]:
df = df.reset_index(drop=True)

In [1402]:
df

,№,Ссылка,Заголовок,Описание задач за 07 декабря (среда),Дата,Тэги
0,4,https://gitlab.mob112.ru/business/supervisor/-...,Расписание маршрутизации,'Необходимо реализовать расписание маршрутизац...,Тверь 15.11.2022,"!Critical,#BackEnd,#FrontEnd,#Phone,*PBX,Требу..."
1,11,https://gitlab.mob112.ru/business/supervisor/-...,Cправочник адресов,"'Нужен ГАР (справочник адресов, с возможностью...",Тверь 15.11.2022,"#BackEnd,#FrontEnd,*Region"
2,34,https://gitlab.mob112.ru/business/supervisor/-...,Нет функционала переадресации звонка,'Отсутствует интерфейс и функционал переадреса...,Тверь 15.11.2022,"#BackEnd,#Design,#FrontEnd,#Phone,#Support,*Re..."
3,38,https://gitlab.mob112.ru/business/supervisor/-...,"Ограничене доступа к функциям Суфлер, конферен...","'Суфлер, конференцию и перехват звонка, разреш...",Тверь 15.11.2022,"#FrontEnd,*Region,Готово на бэке"
4,48,https://gitlab.mob112.ru/business/supervisor/-...,При использовании в пароле символов используем...,'При использовании в пароле пользователя симво...,Тверь 15.11.2022,"!Bugs,#FrontEnd,*Region,Текущий спринт"
5,53,https://gitlab.mob112.ru/business/supervisor/-...,Сортировка карточек обращений,'Кривая сортировка. Закрытая карточка стала пе...,Тверь 15.11.2022,"!Bugs,!Critical,#FrontEnd,*Region,Текущий спринт"
6,57,https://gitlab.mob112.ru/business/supervisor/-...,Не работает перехват звонка,"Не работает перехват звонка между операторами,...",Тверь 15.11.2022,"!Bugs,!Critical,#FrontEnd,#Phone,Текущий спринт"
7,60,https://gitlab.mob112.ru/business/supervisor/-...,Доработки интерфейса анкетирования (survey),'- [x] 1. Требуется визуальное блочное разделе...,Тверь 15.11.2022,"#BackEnd,#Design,#FrontEnd,*Region,*Survey,Сле..."
8,63,https://gitlab.mob112.ru/business/supervisor/-...,Выверить логику отображения и занятости карточ...,'Выверить логику отображения и занятости карто...,Тверь 15.11.2022,"!Bugs,#BackEnd,#FrontEnd,*Region,Текущий сприн..."
9,67,https://gitlab.mob112.ru/business/supervisor/-...,Окно подтверждения получения уведомлений в бра...,'В браузере Chromium при каждом запуске браузе...,Тверь 15.11.2022,"!Bugs,!Critical,#FrontEnd,*Region,Текущий спринт"


In [1403]:
replace_list = [
    '\!.*.png\)',
    '\!.*.jpg\)',
    '\!.*.mov\)',
]

In [1404]:
for replacement in replace_list:
    df = df.replace(to_replace=replacement, value='', regex=True)

In [1405]:
stat = {
    '[x] FrontEnd': '- Фронтенд готов',
    '[x] Front': '- Фронтенд готов',
    '[x] BackEnd': '- Бэкенд готов',
    '[x] Backend': '- Бэкенд готов',
    '[x] Phone': '- Телефония готова',
    '[x] Design': '- Дизайн готов',
    '[ ] FrontEnd ': '- Фронтенд в работе: ',
    '[ ] Frontend ': '- Фронтенд в работе: ',
    '[ ] Front:': '- Фронтенд в работе: ',
    '[ ] FronEnd:': '- Фронтенд в работе: ',
    '[ ] FrontEnd:': '- Фронтенд в работе: ',
    '[ ] BackEnd ': '- Бэкенд в работе: ',
    '[ ] Backend ': '- Бэкенд в работе: ',
    '[ ] BackEnd: ': '- Бэкенд в работе: ',
    '[ ] Backend:': '- Бэкенд в работе: ',
    '[ ] Phone ': '- Телефония в работе: ',
    '[ ] FrontEnd': '- Фронтендер не назначен',
    '[ ] Front': '- Фронтендер не назначен',
    '[ ] BackEnd': '- Бэкендер не назначен',
    '[ ] Phone': '- Телефонист не назначен',
    '[ ] Mobile': '- Специалист мобильной разработки не назначен',
    '[ ] Design': '- Дезайнер не назначен',
    '[ ] Android': '- Разработчик на Android не назначен',
    '[ ] iOS': '- Разработчик на iOS не назначен',
    '[ ] DevOps': '- DevOps не назначен',
}

In [1406]:
for i in stat.items():
    df = df.applymap(lambda x: x.replace(i[0], i[1]) if (isinstance(x, str)) else x)

In [1407]:
empl = {
    '@i.ahmetgareev': 'Ахметгареев Ильмир',
    'Гульшат': 'Губайдуллина Гульшат',
    '@g.gubaydullina': 'Губайдуллина Гульшат',
    '@gaysin': 'Гайсин Тимур',
    '@ArsenMerkin': 'Меркин Арсен',
    '@kuznecov': 'Кузнецов Андрей',
    '@fayzullin': 'Файзуллин Арслан',
    '@nasibullin': 'Насибуллин Тимерьян',
    '@mannanov': 'Маннанов Марат',
    '@halilov': 'Халилов Артур',
    '@stepan': 'Холин Степан',
    '@Rsagadatov': 'Сагадатов Рауль',
    '@boev': 'Боев Егор',
    '@ar.fayzullin': 'Файзуллин Артур'
}

In [1408]:
for i in empl.items():
    df = df.applymap(lambda x: x.replace(i[0], i[1]) if (isinstance(x, str)) else x)

In [1409]:
city = {'Тверь': 'г. Тверь',
        'Брянск': 'г. Брянск'}

In [1410]:
for i in city.items():
    df = df.applymap(lambda x: x.replace(i[0], i[1]) if (isinstance(x, str)) else x)

In [1411]:
df = df.sort_values(by=['№'], ascending=False)

In [1412]:
errors = {
    '!Critical': 'СРОЧНО',
    '!Bugs': 'БАГ',
    'Требуют обсуждения': 'ОБСУДИТЬ',
    '#BackEnd': 'бэкенд',
    '#FrontEnd': 'фронтенд',
    '#Phone': 'телефония',
    '#Design': 'дизайн',
    '*Region': 'регион',
    '*PBX': 'вкц',
    '*Survey': 'опросник',
    'Готово на бэке': 'бэкенд готов',
    'Текущий спринт': '',
    'Следующий спринт': '',
    'Тестирование': 'тестирование',
    'Junior': 'легко',
    'В работе': 'в работе',
}

In [1413]:
for i in errors.items():
    df = df.applymap(lambda x: x.replace(i[0], i[1]) if (isinstance(x, str)) else x)

In [1414]:
fixes = {
    '**': '',
    '\'': '',
    '- [': '[',
    ',': ', ',
    '  ': ' ',
    ', , ': ', ',
    '::': ':',
    '\r\n\r\n\r\n\r\n': '\r\n\r\n',
    '\r\n\r\n': '\r\n',
    '- - ': '- ',
}

In [1415]:
for i in fixes.items():
    df = df.applymap(lambda x: x.replace(i[0], i[1]) if (isinstance(x, str)) else x)

In [1416]:
strip = [
    ', ',
    '\r\n	',
    '\r\n',
    ' ',
]

In [1417]:
for i in strip:
    df = df.applymap(lambda x: x.rstrip(i) if (isinstance(x, str)) else x)

In [1418]:
for i in strip:
    df = df.applymap(lambda x: x.lstrip(i) if (isinstance(x, str)) else x)

In [1419]:
for i in fixes.items():
    df = df.applymap(lambda x: x.replace(i[0], i[1]) if (isinstance(x, str)) else x)

In [1420]:
df = df.applymap(lambda x: x.rstrip('\r\n	') if (isinstance(x, str)) else x)

In [1421]:
df = df.applymap(lambda x: x.lstrip(' ') if (isinstance(x, str)) else x)

In [1422]:
df.fillna('', inplace=True)

In [1423]:
df

,№,Ссылка,Заголовок,Описание задач за 07 декабря (среда),Дата,Тэги
50,136,https://gitlab.mob112.ru/business/supervisor/-...,"При изменении статуса кп на ""завершено"" продол...","- сделать как для других статусов, патчем на и...",,"БАГ, СРОЧНО, фронтенд"
49,135,https://gitlab.mob112.ru/business/supervisor/-...,Ошибка обнуления счетчика поствызывной обработ...,- возможно пропадет если ходить патчем при зав...,,"БАГ, СРОЧНО, фронтенд"
48,134,https://gitlab.mob112.ru/business/supervisor/-...,"Ошибка при попытке изменить карточку ""Expected...",- Фронтенд готов Насибуллин Тимерьян \r\n- htt...,,"БАГ, СРОЧНО, фронтенд"
47,133,https://gitlab.mob112.ru/business/supervisor/-...,Некорректное создание карточки без номера теле...,,,"БАГ, СРОЧНО, фронтенд"
46,132,https://gitlab.mob112.ru/business/supervisor/-...,Ошибка сохранения карточек в системе,- скопировать номер входящего звонка\r\n- пере...,,"БАГ, СРОЧНО, бэкенд, фронтенд"
45,131,https://gitlab.mob112.ru/business/supervisor/-...,Ошибка перехода по заголовку миникарточки звон...,- Переход по заголовку миникарточки звонка по ...,,"БАГ, СРОЧНО, фронтенд"
44,130,https://gitlab.mob112.ru/business/supervisor/-...,Бесконечная загрузка списка карточек,- Фронтендер не назначен,,"БАГ, СРОЧНО, фронтенд"
43,129,https://gitlab.mob112.ru/business/supervisor/-...,Ошибка создания карточки после принятия звонков,"""зависают"" звонки - звонок принят, карточка не...",,"БАГ, СРОЧНО, фронтенд"
42,126,https://gitlab.mob112.ru/business/supervisor/-...,В интерфейсе создания обращения не указывается...,В форме интерфейсе создания обращения не указы...,г. Тверь 29.11.2022,"БАГ, СРОЧНО"
41,125,https://gitlab.mob112.ru/business/supervisor/-...,Вызываемый абонент исходящей компании по обзво...,- Фронтендер не назначен\r\n- Бэкендер не назн...,,"БАГ, СРОЧНО, бэкенд, фронтенд"


In [1424]:
# df.to_csv("/Users/iliya/Egor/iTCODE/reports/output.csv", index=False)

In [1425]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'jsonFileFromGoogle.json', scope)
gc = gspread.authorize(credentials)

In [1426]:
spreadsheet_key = '1hoa2bhnbxmDyCEleUffcE7uzFnE8oWN8myYPywjY0-8'
sheet_id = '1748867294'
wks_name = 'Задачи business'
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'Лист19' id:1316481210>

In [1427]:
def get_service_sacc():
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        'jsonFileFromGoogle.json', scope).authorize(httplib2.Http())
    return build('sheets', 'v4', http=creds)

In [1428]:
sheet = get_service_sacc().spreadsheets()
body = {
    "requests": [

        {
            "repeatCell": {
                "range": {
                    "sheetId": sheet_id,
                },
                "cell": {
                    "userEnteredFormat": {
                        "textFormat": {
                            "fontSize": 14,
                            "fontFamily": "Arial",
                            "bold": False
                        },
                        "horizontalAlignment": "CENTER",
                        "verticalAlignment": "MIDDLE",
                        "wrapStrategy": "WRAP"
                    }
                },
                "fields": "userEnteredFormat"
            },
        },

        {
            "repeatCell": {
                "range": {
                    "sheetId": sheet_id,
                    "startColumnIndex": 3,
                    "endColumnIndex": 4,
                },
                "cell": {
                    "userEnteredFormat": {
                        "textFormat": {
                            "fontSize": 14,
                            "fontFamily": "Arial",
                            "bold": False,
                        },
                        "horizontalAlignment": "LEFT",
                    }
                },
                "fields": "userEnteredFormat(horizontalAlignment)"
            },
        },


        {
            "repeatCell": {
                "range": {
                    "sheetId": sheet_id,
                    "endRowIndex": 1,
                    "endColumnIndex": 6,
                },
                "cell": {
                    "userEnteredFormat": {
                        "backgroundColor": {"red": 0.0, "green": 54.7, "blue": 90.2},
                        "textFormat": {
                            "fontSize": 14,
                            "fontFamily": "Arial",
                            "bold": False,
                            "foregroundColor": {"red": 1.0, "green": 1.0, "blue": 1.0}
                        },
                        "horizontalAlignment": "CENTER",
                    }
                },
                "fields": "userEnteredFormat(backgroundColor, textFormat, horizontalAlignment)"
            },
        },
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_id,
                    "dimension": "COLUMNS",
                    "startIndex": 3,
                    "endIndex": 4,
                },
                "properties" : {
                    "pixelSize": 560
                },
                "fields": "pixelSize",
            }

        },
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_id,
                    "dimension": "COLUMNS",
                    "startIndex": 2,
                    "endIndex": 3,
                },
                "properties" : {
                    "pixelSize": 260
                },
                "fields": "pixelSize",
            }

        }

    ]
}

In [1429]:
response = sheet.batchUpdate(spreadsheetId=spreadsheet_key, body=body).execute()